题目

有`N`种物品和一个容量为`V`的背包，每种物品都有无限件可用。第`i`种物品的费用是`c[i]`，价值是`w[i]`。求解将哪些物品装入背包可使这些物品的费用总和不超过背包容量，且价值总和最大。

从每种物品的角度考虑，与它相关的策略已并非取或不取两种，而是有取0件、取1件、取2件……等很多种。如果仍然按照解01背包时的思路，令f[i][v]表示前i种物品恰放入一个容量为v的背包的最大权值。仍然可以按照每种物品不同的策略写出状态转移方程，像这样：
>$$f[i][v]=max\{f[i-1][v-k*c[i]]+k*w[i]|0<=k*c[i]<=v\}$$

这跟01背包问题一样有$O(VN)$个状态需要求解，但求解每个状态的时间已经不是常数了，求解状态$f[i][v]$的时间是$O(v/c[i])$，总的复杂度可以认为是$O(V*Σ(V/c[i]))$，是比较大的。

## 一个简单有效的优化

完全背包问题有一个很简单有效的优化，是这样的：若两件物品`i、j`满足`c[i]=w[j]`，则将物品j去掉，不用考虑。这个优化的正确性显然：任何情况下都可将价值小费用高得`j`换成物美价廉的`i`，得到至少不会更差的方案。对于随机生成的数据，这个方法往往会大大减少物品的件数，从而加快速度。然而这个并不能改善最坏情况的复杂度，因为有可能特别设计的数据可以一件物品也去不掉。

这个优化可以简单的$O(N^2)$地实现，一般都可以承受。

另外，针对背包问题而言，比较不错的一种方法是：首先将费用大于$V$的物品去掉，然后使用类似计数排序的做法，计算出费用相同的物品中价值最高的是哪个，可以$O(V+N)$地完成这个优化。这个不太重要的过程就不给出伪代码了，希望你能独立思考写出伪代码或程序。

## 转化为01背包问题求解
既然`01`背包问题是最基本的背包问题，那么我们可以考虑把完全背包问题转化为`01`背包问题来解。最简单的想法是，考虑到第`i`种物品最多选$V/c[i]$件，于是可以把第`i`种物品转化为$V/c[i]$件费用及价值均不变的物品，然后求解这个`01`背包问题。这样完全没有改进基本思路的时间复杂度，但这毕竟给了我们将完全背包问题转化为01背包问题的思路：将一种物品拆成多件物品。

更高效的转化方法是：把第`i`种物品拆成费用为$c[i]*2^k$、价值为$w[i]*2^k$的若干件物品，其中$k$满足$c[i]*2^k<=V$。这是二进制的思想，因为不管最优策略选几件第$i$种物品，总可以表示成若干个$2^k$件物品的和。这样把每种物品拆成$O(log V/c[i])$件物品，是一个很大的改进。

但我们有更优的O(VN)的算法。



## O(VN)的算法
这个算法使用一维数组，先看伪代码：
```
for i=1..N
    for v=0..V
        f[v]=max{f[v],f[v-cost]+weight}
```
这个伪代码与[01背包](./pack_01.md)的伪代码只有v的循环次序不同而已。为什么这样一改就可行呢？首先想想为什么P01中要按照$v=V..0$的逆序来循环。这是因为要保证第$i$次循环中的状态$f[i][v]$是由状态$f[i-1][v-c[i]]$递推而来。换句话说，这正是为了保证每件物品只选一次，保证在考虑“选入第i件物品”这件策略时，依据的是一个绝无已经选入第$i$件物品的子结果$f[i-1][v-c[i]]$。而现在完全背包的特点恰是每种物品可选无限件，所以在考虑“加选一件第$i$种物品”这种策略时，却正需要一个可能已选入第i种物品的子结果$f[i][v-c[i]]$，所以就可以并且必须采用$v=0..V$的顺序循环。这就是这个简单的程序为何成立的道理。

### 优化

伪代码中两层for循环的次序可以颠倒。这个结论有可能会带来算法时间常数上的优化。

这个算法也可以以另外的思路得出。例如，将基本思路中求解$f[i][v-c[i]]$的状态转移方程显式地写出来，代入原方程中，会发现该方程可以等价地变形成这种形式：
> $f[i][v]=max{f[i-1][v],f[i][v-c[i]]+w[i]}$

将这个方程用一维数组实现，便得到了上面的伪代码。

最后抽象出处理一件完全背包类物品的过程伪代码：
```
procedure CompletePack(cost,weight)
    for v=cost..V
        f[v]=max{f[v],f[v-c[i]]+w[i]}
```